# Configuration Access

<b>This notebook is not supposed to be used on its own.<b>

In [ ]:
def get_access_store_ui(root_dir: str = '.'):
    
    from pathlib import Path
    import ipywidgets as widgets
    from exasol.secret_store import Secrets as Secrets_
    
    # A fix for the secret store to make its interface more in line with the dict.
    class Secrets(Secrets_):
        
        def get(self, key: str, default_value: str=None) -> str:
            value = super().get(key)
            return default_value if value is None else value

        def __getattr__(self, key):
            value = self.get(key)
            if value is None:
                raise AttributeError(f'{key} value is not defined')
            return value

    # Try to find the file name and the password in the shared store.
    # Create global variables only temporarily.
    %store -r
    if 'sb_store_file' in globals():
        global sb_store_file
        sb_store_file_ = sb_store_file
        del sb_store_file
    else:
        sb_store_file_ = 'dss_config.sqlite'
    if 'sb_password' in globals():
        global sb_password
        sb_password_ = sb_password
        del sb_password
    else:
        sb_password_ = ''

    style = {'description_width': '120px'}
    inputs = {
        'file':     widgets.Text(description='Config. File Name', value=sb_store_file_, style=style),
        'password': widgets.Password(description='Config. Password', value=sb_password_, style=style),
        'ok':       widgets.Button(description='OK')
    }

    def open_or_create_config_store(btn):
        global sb_config, sb_store_file, sb_password
        sb_store_file = inputs['file'].value
        sb_password = inputs['password'].value
        sb_config = Secrets(Path(root_dir) / sb_store_file, sb_password)
        # Save the file and password in the shared store.
        %store sb_store_file sb_password
        del sb_store_file, sb_password
    
    inputs['ok'].on_click(open_or_create_config_store)
    
    ui = widgets.VBox(list(inputs.values()))
    return ui